In [ ]:
pip install pandas

In [ ]:
import os
import pandas as pd

# Path to the main MIMIC-IV folder
data_path = "MIMIC_IV"

# Find all CSV files in the dataset
csv_files = {}
for root, _, files in os.walk(data_path):
    for file in files:
        if file.endswith(".csv"):
            file_path = os.path.join(root, file)
            csv_files[file] = pd.read_csv(file_path)

# Print loaded files
print(f"Loaded {len(csv_files)} CSV files:")
print(csv_files.keys())

In [ ]:
import pandas as pd

# Replace these with the actual file paths for your CSV files.
edstays = pd.read_csv("MIMIC_IV_datasets/edstays.csv")
diagnosis = pd.read_csv("MIMIC_IV_datasets/diagnosis.csv")
medrecon = pd.read_csv("MIMIC_IV_datasets/medrecon.csv")
pyxis = pd.read_csv("MIMIC_IV_datasets/pyxis.csv")
triage = pd.read_csv("MIMIC_IV_datasets/triage.csv")
vitalsign = pd.read_csv("MIMIC_IV_datasets/vitalsign.csv")

In [ ]:
print("EDStays:", edstays.shape)
print("Diagnosis:", diagnosis.shape)
print("MedRecon:", medrecon.shape)
print("Pyxis:", pyxis.shape)
print("Triage:", triage.shape)
print("VitalSign:", vitalsign.shape)

**Medrecon Data pre-processing**

In [ ]:
import pandas as pd

# Count the rows where gsn is 0
zero_gsn_count = medrecon[medrecon['gsn'] == 0].shape[0]
print("Number of rows with gsn value 0:", zero_gsn_count)


In [ ]:
import numpy as np

medrecon['gsn'] = medrecon['gsn'].replace(0, np.nan)
medrecon['ndc'] = medrecon['ndc'].replace(0, np.nan)

**Converting to Lower-case to make standard format**

In [ ]:
medrecon['name'] = medrecon['name'].str.strip().str.lower()

In [ ]:
medrecon['charttime'] = pd.to_datetime(medrecon['charttime'])

In [ ]:
medrecon.head()

**Edstays data pre-processing**

In [ ]:
edstays.head()

In [ ]:
diagnosis.head()

In [ ]:
pyxis.head()

In [ ]:
triage.head()

In [ ]:
merged_df1 = pd.merge(edstays, diagnosis, on=["subject_id", "stay_id"], how="outer")
merged_df1.head()

In [ ]:
merged_df2 = pd.merge(merged_df1, triage, on=["subject_id", "stay_id"], how="outer")
merged_df2.head()

In [ ]:
final_df = pd.merge(merged_df2, medrecon, on=["subject_id", "stay_id"], how="outer")
final_df.head()

In [ ]:
final_df.shape

In [ ]:
final_df.columns

In [ ]:
# List of columns to drop
columns_to_drop = [    'intime', 'outtime', 'race', 'ndc', 'etc_rn', 'etccode',
                   'arrival_transport', 'disposition', 'icd_version', 'seq_num', 'pain', 'acuity', 'charttime']

# Dropping the columns from the DataFrame
df = final_df.drop(columns=columns_to_drop)

df.head()

In [ ]:
df.shape

In [ ]:
df.columns

In [ ]:
# Find unique ICD codes in the dataset
unique_icd_codes = df['icd_code'].unique()

# Display the unique ICD codes
print(unique_icd_codes)



In [ ]:
# Count the number of unique ICD codes
num_unique_icd_codes = df['icd_code'].nunique()

# Display the count
print(f"Number of unique ICD codes: {num_unique_icd_codes}")


In [ ]:
# Count the frequency of each ICD code
icd_code_counts = df['icd_code'].value_counts()

# Display the frequency of ICD codes
print(icd_code_counts)


In [ ]:
df['icd_category'] = df['icd_code'].str[:3]  # First 3 characters

In [ ]:
df['icd_category']

In [ ]:
df.head()

In [ ]:
df.shape

In [ ]:
df.to_csv("merged_mimic_IV.csv", index=False)
print("Merged data saved successfully!")

In [ ]:
import pandas as pd

# Load your final dataset
final_df = pd.read_csv("merged_mimic_IV.csv")  # Replace with your actual file path

# Display the first few rows
final_df.head()


In [ ]:
import pandas as pd

column_names = ["chapter", "subcode", "icd_code", "long_desc", "short_desc", "category"]
icd_df = pd.read_csv("ICD10codes.csv", names=column_names, header=None)


In [ ]:
icd_df.head()

In [ ]:
icd_df.to_csv("icd_codes.csv", index=False)
print("icd codes with column names saved successfully!")

In [ ]:
# Load your final dataset
icd_df = pd.read_csv("icd_codes.csv")  # Replace with your actual file path

# Display the first few rows
icd_df.head()

In [ ]:
# Merge the data on the icd_code column
merged_df = final_df.merge(icd_df, on="icd_code", how="left")

# Display a few rows to check the result
merged_df.head()


In [ ]:
merged_df.columns

In [ ]:
# Count unique admissions (or rows) per ICD category
grouped_by_icd_category = merged_df.groupby('icd_category').agg({
    'hadm_id': 'nunique'  # or 'count' if you want the total rows per category
}).rename(columns={'hadm_id': 'unique_admissions'})

print("Grouped by ICD Category:")
grouped_by_icd_category


In [ ]:
# Group by chapter and count unique admissions
grouped_by_chapter = merged_df.groupby('chapter').agg({'hadm_id': 'nunique'}).rename(columns={'hadm_id': 'unique_admissions'})

# Sum of all grouped rows should equal the total number of rows in the DataFrame
total_grouped_rows = grouped_by_chapter.sum()
total_rows = merged_df.shape[0]

print("Total rows in grouping:", total_grouped_rows)
print("Total rows in DataFrame:", total_rows)

In [ ]:
# Count rows where the chapter value is missing (NaN)
missing_chapter_count = merged_df['chapter'].isnull().sum()

# Count rows where the chapter value is an empty string (if applicable)
empty_chapter_count = (merged_df['chapter'].astype(str).str.strip() == "").sum()

print("Rows with missing chapter (NaN):", missing_chapter_count)
print("Rows with empty chapter:", empty_chapter_count)


In [ ]:
empty_subject_id = (merged_df['subject_id']).isna().sum().sum()

print("Rows with empty Subject_id:", empty_subject_id)

In [ ]:
missing_icd_code_count = merged_df['icd_code'].isnull().sum()

empty_icd_code = (merged_df['icd_code'].astype(str).str.strip() == "").sum()

print("Rows with missing icd_code (NaN):", missing_icd_code_count)
print("Rows with empty icd_code:", empty_icd_code)

In [ ]:
import pandas as pd

def reformat_icd9(icd_str):

    # If a dot is already present, return as is.
    if '.' in icd_str:
        return icd_str
    # Remove any leading/trailing whitespace.
    icd_str = icd_str.strip()
    # For codes with 3 or more digits, insert a dot after the third digit.
    if len(icd_str) > 2:
        return icd_str[:3] + '.' + icd_str[3:]
    else:
        return icd_str

def infer_icd_chapter(icd_code):
    """
    Infers the ICD chapter (broad category) for a given ICD code,
    handling both ICD-10 (alphabetic codes) and ICD-9 (numeric codes).
    For ICD-9 codes without a decimal, the function inserts one before conversion.
    """
    if pd.isna(icd_code):
        return None

    # Convert code to string and strip whitespace
    icd_str = str(icd_code).strip()
    if not icd_str:
        return 'Other'

    # Special handling: if code starts with "R" followed by digits, process the numeric part
    if icd_str[0].upper() == 'R' and icd_str[1:].replace('.', '', 1).isdigit():
        numeric_part = icd_str[1:]
        # Optionally reformat the numeric part if needed (using your helper function)
        icd_str_reformatted = reformat_icd9(numeric_part)
        try:
            code_num = float(icd_str_reformatted.split('.')[0])
        except ValueError:
            return 'Other'

        # Use ICD-9 numeric ranges for categorization
        if 1 <= code_num <= 139:
            return 'Infectious and parasitic diseases'
        elif 140 <= code_num <= 239:
            return 'Neoplasms'
        elif 240 <= code_num <= 279:
            return 'Endocrine, nutritional and metabolic diseases'
        elif 280 <= code_num <= 289:
            return 'Diseases of the blood and blood-forming organs'
        elif 290 <= code_num <= 319:
            return 'Mental disorders'
        elif 320 <= code_num <= 389:
            return 'Diseases of the nervous system and sense organs'
        elif 390 <= code_num <= 459:
            return 'Diseases of the circulatory system'
        elif 460 <= code_num <= 519:
            return 'Diseases of the respiratory system'
        elif 520 <= code_num <= 579:
            return 'Diseases of the digestive system'
        elif 580 <= code_num <= 629:
            return 'Diseases of the genitourinary system'
        elif 630 <= code_num <= 677:
            return 'Complications of pregnancy, childbirth and the puerperium'
        elif 680 <= code_num <= 709:
            return 'Diseases of the skin and subcutaneous tissue'
        elif 710 <= code_num <= 739:
            return 'Diseases of the musculoskeletal system and connective tissue'
        elif 740 <= code_num <= 759:
            return 'Congenital anomalies'
        elif 760 <= code_num <= 779:
            return 'Certain conditions originating in the perinatal period'
        elif 780 <= code_num <= 789:
            if code_num == 780:
                return "General symptoms"
            elif code_num == 781:
                return "Symptoms involving nervous and musculoskeletal systems"
            elif code_num == 782:
                return "Symptoms involving skin and other integumentary tissue"
            elif code_num == 783:
                return "Symptoms concerning nutrition metabolism and development"
            elif code_num == 784:
                return "Symptoms involving head and neck"
            elif code_num == 785:
                return "Symptoms involving cardiovascular system"
            elif code_num == 786:
                return "Symptoms involving respiratory system and other chest symptoms"
            elif code_num == 787:
                return "Symptoms involving digestive system"
            elif code_num == 788:
                return "Symptoms involving urinary system"
            elif code_num == 789:
                return "Other symptoms involving abdomen and pelvis"
        elif 790 <= code_num <= 796:
            if code_num == 790:
                return "Nonspecific findings on examination of blood"
            elif code_num == 791:
                return "Nonspecific findings on examination of urine"
            elif code_num == 792:
                return "Nonspecific abnormal findings in other body substances"
            elif code_num == 793:
                return "Nonspecific abnormal (findings) on radiological and other examination of body structure"
            elif code_num == 794:
                return "Nonspecific abnormal results of function studies"
            elif code_num == 795:
                return "Other and nonspecific abnormal cytological, histological, immunological, and dna test findings"
            elif code_num == 796:
                return "Other nonspecific abnormal findings"
        elif 797 <= code_num <= 799:
            if code_num == 797:
                return "Senility without mention of psychosis"
            elif code_num == 798:
                return "Sudden death cause unknown"
            elif code_num == 799:
                return "Other ill-defined and unknown causes of morbidity and mortality"
        elif 800 <= code_num <= 999:
            return 'Injury and poisoning'
        else:
            return 'Other'

    # Check if it's ICD-10 (alphabetic)
    if icd_str[0].isalpha():
        mapping_icd10 = {
            'A': 'Certain infectious and parasitic diseases',
            'B': 'Certain infectious and parasitic diseases',
            'C': 'Neoplasms',
            'D': 'Neoplasms / Blood diseases',  # Adjust as needed
            'E': 'Endocrine, nutritional and metabolic diseases',
            'F': 'Mental and behavioural disorders',
            'G': 'Diseases of the nervous system',
            'H': 'Diseases of the eye and adnexa',
            'I': 'Diseases of the circulatory system',
            'J': 'Diseases of the respiratory system',
            'K': 'Diseases of the digestive system',
            'L': 'Diseases of the skin and subcutaneous tissue',
            'M': 'Diseases of the musculoskeletal system and connective tissue',
            'N': 'Diseases of the genitourinary system',
            'O': 'Pregnancy, childbirth and the puerperium',
            'P': 'Certain conditions originating in the perinatal period',
            'Q': 'Congenital malformations, deformations and chromosomal abnormalities',
            'R': 'Symptoms, signs and abnormal clinical findings',
            'S': 'Injury, poisoning and certain other consequences of external causes',
            'T': 'Injury, poisoning and certain other consequences of external causes',
            'V': 'External causes of morbidity and mortality',
            'W': 'External causes of morbidity and mortality',
            'X': 'External causes of morbidity and mortality',
            'Y': 'External causes of morbidity and mortality',
            'Z': 'Factors influencing health status and contact with health services'
        }
        return mapping_icd10.get(icd_str[0], 'Other')
    else:
        # It's ICD-9. Reformat if necessary.
        icd_str_reformatted = reformat_icd9(icd_str)
        try:
            # Convert the part before any decimal to a float.
            code_num = float(icd_str_reformatted.split('.')[0])
        except ValueError:
            return 'Other'

        # Map numeric ranges according to common ICD-9 groupings:
        if 1 <= code_num <= 139:
            return 'Infectious and parasitic diseases'
        elif 140 <= code_num <= 239:
            return 'Neoplasms'
        elif 240 <= code_num <= 279:
            return 'Endocrine, nutritional and metabolic diseases'
        elif 280 <= code_num <= 289:
            return 'Diseases of the blood and blood-forming organs'
        elif 290 <= code_num <= 319:
            return 'Mental disorders'
        elif 320 <= code_num <= 389:
            return 'Diseases of the nervous system and sense organs'
        elif 390 <= code_num <= 459:
            return 'Diseases of the circulatory system'
        elif 460 <= code_num <= 519:
            return 'Diseases of the respiratory system'
        elif 520 <= code_num <= 579:
            return 'Diseases of the digestive system'
        elif 580 <= code_num <= 629:
            return 'Diseases of the genitourinary system'
        elif 630 <= code_num <= 677:
            return 'Complications of pregnancy, childbirth and the puerperium'
        elif 680 <= code_num <= 709:
            return 'Diseases of the skin and subcutaneous tissue'
        elif 710 <= code_num <= 739:
            return 'Diseases of the musculoskeletal system and connective tissue'
        elif 740 <= code_num <= 759:
            return 'Congenital anomalies'
        elif 760 <= code_num <= 779:
            return 'Certain conditions originating in the perinatal period'
        elif 780 <= code_num <= 789:
            # Symptoms (ICD-9 780–789)
            if code_num == 780:
                return "General symptoms"
            elif code_num == 781:
                return "Symptoms involving nervous and musculoskeletal systems"
            elif code_num == 782:
                return "Symptoms involving skin and other integumentary tissue"
            elif code_num == 783:
                return "Symptoms concerning nutrition metabolism and development"
            elif code_num == 784:
                return "Symptoms involving head and neck"
            elif code_num == 785:
                return "Symptoms involving cardiovascular system"
            elif code_num == 786:
                return "Symptoms involving respiratory system and other chest symptoms"
            elif code_num == 787:
                return "Symptoms involving digestive system"
            elif code_num == 788:
                return "Symptoms involving urinary system"
            elif code_num == 789:
                return "Other symptoms involving abdomen and pelvis"
        elif 790 <= code_num <= 796:
            # Nonspecific abnormal findings (ICD-9 790–796)
            if code_num == 790:
                return "Nonspecific findings on examination of blood"
            elif code_num == 791:
                return "Nonspecific findings on examination of urine"
            elif code_num == 792:
                return "Nonspecific abnormal findings in other body substances"
            elif code_num == 793:
                return "Nonspecific abnormal (findings) on radiological and other examination of body structure"
            elif code_num == 794:
                return "Nonspecific abnormal results of function studies"
            elif code_num == 795:
                return "Other and nonspecific abnormal cytological, histological, immunological, and dna test findings"
            elif code_num == 796:
                return "Other nonspecific abnormal findings"
        elif 797 <= code_num <= 799:
            # Ill-defined and unknown causes of morbidity and mortality (ICD-9 797–799)
            if code_num == 797:
                return "Senility without mention of psychosis"
            elif code_num == 798:
                return "Sudden death cause unknown"
            elif code_num == 799:
                return "Other ill-defined and unknown causes of morbidity and mortality"
        elif 800 <= code_num <= 999:
            return 'Injury and poisoning'
        else:
            return 'Other'

# Apply the function to your merged DataFrame (assuming the ICD code column is named 'icd_code')
merged_df['icd_chapter_combined'] = merged_df['icd_code'].apply(infer_icd_chapter)

# Check the distribution of the combined ICD chapters
print(merged_df['icd_chapter_combined'].value_counts())


In [ ]:
import matplotlib.pyplot as plt

# Plot the distribution of ICD categories
category_counts = merged_df['icd_chapter_combined'].value_counts()
plt.figure(figsize=(10, 6))
category_counts.plot(kind='bar', color='skyblue')
plt.title('ICD Chapter Distribution')
plt.xlabel('ICD Chapter')
plt.ylabel('Count')
plt.xticks(rotation=90)
plt.show()

In [ ]:
import matplotlib.pyplot as plt

# Sample gender distribution data (replace with actual data)
gender_counts = merged_df['gender'].value_counts()

# Plot
fig, ax = plt.subplots(figsize=(6, 4))
bars = gender_counts.plot(kind='bar', ax=ax, color=['blue', 'pink'])

# Add labels on top of bars
for bar in bars.patches:
    yval = bar.get_height()
    ax.text(bar.get_x() + bar.get_width()/2, yval + 10, int(yval), ha='center', va='bottom')

ax.set_title("Gender Distribution")
ax.set_xlabel("Gender")
ax.set_ylabel("Count")

plt.show()


In [ ]:
merged_df['chiefcomplaint'].value_counts().head(10)

In [ ]:
from wordcloud import WordCloud

text = " ".join(merged_df['chiefcomplaint'].dropna())
wordcloud = WordCloud(width=800, height=400, background_color="white").generate(text)

plt.figure(figsize=(10, 5))
plt.imshow(wordcloud, interpolation="bilinear")
plt.axis("off")
plt.show()


In [ ]:
import matplotlib.pyplot as plt

# Define vitals list
vitals = ['temperature', 'heartrate', 'resprate', 'o2sat', 'sbp', 'dbp']

# Define optimal x-axis ranges for each vital based on typical clinical values
vital_ranges = {
    'temperature': (30, 45),   # Normal: 36-37.5°C, but allowing wider range for outliers
    'heartrate': (30, 200),    # Normal: 60-100 bpm
    'resprate': (5, 50),       # Normal: 12-20 breaths/min
    'o2sat': (50, 100),        # Normal: 95-100%
    'sbp': (50, 250),          # Normal: 90-120 mmHg
    'dbp': (30, 150)           # Normal: 60-80 mmHg
}

# Define y-axis limit dynamically based on histogram frequencies
fig, axes = plt.subplots(nrows=2, ncols=3, figsize=(15, 10))
axes = axes.flatten()

for i, vital in enumerate(vitals):
    ax = axes[i]
    
    # Drop NaN values
    data = merged_df[vital].dropna()
    
    # Plot histogram with adjusted x-axis limits
    counts, bins, patches = ax.hist(data, bins=50, range=vital_ranges[vital], alpha=0.75, color='blue')
    
    # Adjust y-axis limit dynamically
    ax.set_ylim(0, max(counts) * 1.1)
    
    ax.set_title(f"{vital.capitalize()} Distribution")
    ax.set_xlabel(vital.capitalize())
    ax.set_ylabel("Frequency")

plt.tight_layout()
plt.show()


In [ ]:
# Sepsis or Severe Infection Risk - ICU patients with high fever + high heart rate + low BP
sepsis_patients = merged_df[
    (merged_df['temperature'] > 38.5) & 
    (merged_df['heartrate'] > 110) & 
    (merged_df['sbp'] < 90) &
    (merged_df['stay_id'].notna())  # Ensure the patient has a valid stay_id
]
print(f"Potential Sepsis Patients in ICU: {len(sepsis_patients)}")

In [ ]:
import matplotlib.pyplot as plt

# Check if mimic_df is defined before running this
if 'merged_df' in locals():
    # Get top 10 prescribed medications
    top_medications = merged_df['name'].value_counts().head(10)

    # Plot the bar chart
    fig, ax = plt.subplots(figsize=(10, 5))
    bars = top_medications.plot(kind='bar', ax=ax, color='skyblue')

    # Add labels on top of bars
    for bar in bars.patches:
        yval = bar.get_height()
        ax.text(bar.get_x() + bar.get_width()/2, yval + 5, int(yval), ha='center', va='bottom')

    ax.set_title("Top 10 Prescribed Medications")
    ax.set_xlabel("Drug")
    ax.set_ylabel("Count")

    plt.xticks(rotation=45)
    plt.show()
else:
    print("Error: mimic_df is not defined. Please load the dataset first.")

In [ ]:
print(circulatory_df['icd_title'].value_counts().head(20))  # Show top 20 circulatory disease names


In [ ]:
plt.figure(figsize=(12, 6))
sns.violinplot(x="icd_title", y="heartrate", data=filtered_df)
plt.xticks(rotation=45)
plt.title("Heart Rate Distribution by ICD Diagnosis")
plt.xlabel("ICD Title")
plt.ylabel("Heart Rate")
plt.show()

In [ ]:
# Bar Plot: Mean Vitals per Diagnosis (Fixing X-Axis Labels)
import seaborn as sns
import matplotlib.pyplot as plt

# Compute the average vitals per diagnosis (Top 10 diagnoses for readability)
top_icd_titles = merged_df['icd_title'].value_counts().index[:10]  # Select top 10 most common ICD titles
filtered_df = merged_df[merged_df['icd_title'].isin(top_icd_titles)]
avg_vitals = filtered_df.groupby('icd_title')[['heartrate', 'sbp', 'dbp', 'o2sat', 'resprate', 'temperature']].mean().reset_index()

# Plot bar chart for a selected vital sign (e.g., Heart Rate)
plt.figure(figsize=(12, 6))
ax = sns.barplot(x="icd_title", y="heartrate", data=avg_vitals, dodge=False)

# Rotate x-axis labels for better alignment
ax.set_xticklabels(ax.get_xticklabels(), rotation=45, ha="right")

# Add title and labels
plt.title("Average Heart Rate by ICD Diagnosis")
plt.xlabel("ICD Title")
plt.ylabel("Heart Rate")

plt.show()

Clean ICD_Title

In [ ]:
import pandas as pd
import re

# Function to clean icd_title by removing 'unspecified' and special characters
def clean_icd_title(title):
    if pd.isna(title):  # Check for NaN values
        return ""
    title = re.sub(r',?\s*unspecified', '', title, flags=re.IGNORECASE)  # Remove 'unspecified' with optional comma
    title = re.sub(r'[^a-zA-Z0-9\s]', '', title)  # Remove all special characters except spaces
    return title.strip()  # Remove leading/trailing spaces

# Apply the function to clean 'icd_title'
merged_df['icd_title'] = merged_df['icd_title'].apply(clean_icd_title)

# Display unique cleaned values to verify
#print(merged_df['icd_title'].unique())


In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

# Ensure 'icd_title' column is cleaned by removing 'unspecified' diagnoses
merged_df['icd_title'] = merged_df['icd_title'].str.replace(r'unspecified', '', case=False, regex=True).str.strip()

# Select the top 10 most common diagnoses after cleaning
top_diagnoses = merged_df['icd_title'].value_counts().index[:10]

# Filter dataset for only these diagnoses
filtered_df = merged_df[merged_df['icd_title'].isin(top_diagnoses)]

# Count occurrences of each diagnosis by gender
diagnosis_gender_counts = filtered_df.groupby(['icd_title', 'gender']).size().unstack()

# Define custom colors for Male & Female
custom_colors = ['#1f77b4', '#ff7f0e']  # Blue for Male, Orange for Female

# Plot stacked bar chart with custom colors
fig, ax = plt.subplots(figsize=(12, 6))
diagnosis_gender_counts.plot(kind='bar', stacked=True, color=custom_colors, ax=ax)

# Formatting
plt.title("Diagnosis Distribution by Gender (Top 10 Diagnoses)")
plt.xlabel("ICD Title (Diagnosis)")
plt.ylabel("Number of Cases")
plt.xticks(rotation=45, ha='right')
plt.legend(title="Gender")
plt.grid(axis='y', linestyle='--', alpha=0.7)

# Add value labels on top of bars
for container in ax.containers:
    ax.bar_label(container, fmt='%d', label_type='center', fontsize=10, color='white')

# Show plot
plt.show()
